In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

In [ ]:
df = pd.read_csv('../../data/preprocessed_data/시도_시군구별_보행자_사고_사고유형_전처리ver.csv')

# 피해 가중 점수 계산
df['피해가중점수'] = (
    df['사망자수'] * 1 +
    df['중상자수'] * 0.1 +
    df['경상자수'] * 0.01 +
    df['부상신고자수'] * 0.001
)

# 가중치 계산
avg_by_area = df.groupby('시군구')['피해가중점수'].mean().to_dict()
type_weight = df.groupby('사고유형')['피해가중점수'].mean()
road_weight = df.groupby('도로형태')['피해가중점수'].mean()

# feature 생성
df['사고도로조합'] = df['사고유형'] + '_' + df['도로형태']
df['시군구_평균사고'] = df['시군구'].map(avg_by_area)
df['사고유형가중치'] = df['사고유형'].map(type_weight)
df['도로형태가중치'] = df['도로형태'].map(road_weight)

df['최종위험점수'] = df['피해가중점수'] + 0.5 * (df['사고유형가중치'] + df['도로형태가중치'])

# 시군구별 평균 위험도 기반 고위험 지역 여부 정의
avg_risk_by_area = df.groupby('시군구')['최종위험점수'].mean()
threshold = avg_risk_by_area.quantile(0.8)

df['시군구_위험여부'] = df['시군구'].map(lambda x: int(avg_risk_by_area[x] > threshold))

feature_cols = ['사고유형', '도로형태', '피해가중점수', '사고도로조합', '시군구_평균사고']      # feature 선택
target_col = '시군구_위험여부'                                                       # target 선택

df_model = df[feature_cols + [target_col]].dropna()                              # 결측 제거

train_data, test_data = train_test_split(df_model, test_size=0.2, random_state=42, stratify=df_model[target_col])   # 데이터 분할

# 모델 학습
predictor = TabularPredictor(label=target_col, path='HighRiskClassifier', problem_type='binary').fit(
    train_data=train_data,
    presets='high_quality',
    time_limit=600,
    num_cpus=10,
    verbosity=2
)

predictor.save()                # 모델 저장
predictor.evaluate(test_data)   # 모델 평가

# 리더보드 출력
leaderboard = predictor.leaderboard(test_data, silent=True)
print(leaderboard)

In [ ]:
# 예측 예시
sample = pd.DataFrame([{
    '사고유형': '차대사람 - 차도통행중',
    '도로형태': '교차로 - 교차로부근',
    '피해가중점수': 1.15,
    '사고도로조합': '차대사람 - 차도통행중_교차로 - 교차로부근',
    '시군구_평균사고': avg_by_area.get('종로구', 0)
}])
print(predictor.predict(sample))